# CS6405 - Data Mining - First Assignment

Lecturer: Dr Andrea Visentin


## Submission

This assignment is **due on 03/03/24 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for **25 Marks** of your module grade.

## Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism and I agree to the
requirements set out thereby in relation to plagiarism and referencing. I confirm that I
have referenced and acknowledged properly all sources used in preparation of this
assignment.
I declare that this assignment is entirely my own work based on my personal study. I
further declare that I have not engaged the services of another to either assist me in, or
complete this assignment”</font>

## Specification

The objective of this project is to build a k-Nearest Neighbour algorithm that takes as input training and test dataset and will predict the target binary variable with a reasonable degree of accuracy.

You will find a template for your python code in Canvas along with the dataset. In this project, you must implement your own machine learning library and therefore **you are only allowed to use the following python packages and libraries: NumPy, Pandas.**

This project focuses on the Titanic dataset that we used during the lectures. You can find the dataset at:

https://github.com/andvise/DataAnalyticsDatasets/blob/main/titanic.csv


# Tasks:



# Data Preparation (5 marks)



1.  Load the dataset on Colab


In [136]:
# YOUR CODE HERE
import pandas as pd
import numpy as np

df=pd.read_csv("https://raw.githubusercontent.com/andvise/DataAnalyticsDatasets/main/titanic.csv")
#df = df.sample(frac=1, random_state=0).reset_index(drop=True)
df.head()
df.shape

(887, 9)



2. Display the attributes' name and their data type




In [137]:
# YOUR CODE HERE
df.dtypes

PassengerId                  int64
Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard    float64
Fare                       float64
dtype: object

3. Delete the columns that should not be included in a k-NN technique. Explain why you removed them in 1-2 sentences.




In [138]:
len(df['Name'].unique())   #This shows all the values in Name column is unique

887

In [139]:
# YOUR CODE HERE

df.drop(['Name','PassengerId'],inplace=True,axis=1)


# PassengerId has no specific correlation with survived column as it is acting as an index hence dropped.
# Name has object data type and is unique for all the passengers in the dataset hence removing it rather than encoding would be a better idea.

In [140]:
df

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,male,22.0,1,0.0,7.2500
1,1,1,female,38.0,1,0.0,71.2833
2,1,3,female,26.0,0,0.0,7.9250
3,1,1,female,35.0,1,NaN,53.1000
4,0,3,male,35.0,0,0.0,8.0500
...,...,...,...,...,...,...,...
882,0,2,male,27.0,0,0.0,13.0000
883,1,1,female,19.0,0,0.0,30.0000
884,0,3,female,7.0,1,2.0,23.4500
885,1,1,male,26.0,0,0.0,30.0000


4. Replace all missing values with 0


In [141]:
df.isnull().sum()


Survived                   0
Pclass                     0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    1
Fare                       0
dtype: int64

In [142]:
# YOUR CODE HERE

df.fillna(0,inplace=True)
df.isnull().sum()

Survived                   0
Pclass                     0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64

5. Transform the Sex column into a numerical one

In [143]:
# YOUR CODE HERE
l=[]
for i in df['Sex']:
  if i=='male':
    l.append(0)
  else:
    l.append(1)

df['Sex_t']=l
print(df['Sex_t'].value_counts())
df.Sex.value_counts()

0    573
1    314
Name: Sex_t, dtype: int64


male      573
female    314
Name: Sex, dtype: int64

In [144]:
df.drop(['Sex'],axis=1,inplace=True)

In [145]:
df.head()

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_t
0,0,3,22.0,1,0.0,7.2500,0
1,1,1,38.0,1,0.0,71.2833,1
2,1,3,26.0,0,0.0,7.9250,1
3,1,1,35.0,1,0.0,53.1000,1
4,0,3,35.0,0,0.0,8.0500,0


6. Use **Survived** as the target label and the rest of the data frame as features

In [146]:
# YOUR CODE HERE
target=df['Survived']
features=df.drop(['Survived'],axis=1)
features

,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_t
0,3,22.0,1,0.0,7.2500,0
1,1,38.0,1,0.0,71.2833,1
2,3,26.0,0,0.0,7.9250,1
3,1,35.0,1,0.0,53.1000,1
4,3,35.0,0,0.0,8.0500,0
...,...,...,...,...,...,...
882,2,27.0,0,0.0,13.0000,0
883,1,19.0,0,0.0,30.0000,1
884,3,7.0,1,2.0,23.4500,1
885,1,26.0,0,0.0,30.0000,0


5. Divide your dataset into 80% for training and 20% for test


In [147]:
# YOUR CODE HERE
split_index = int(len(df) * 0.8)
x_train = features.iloc[:split_index]
y_train = target.iloc[:split_index]

x_test=features.iloc[split_index:]
y_test=target.iloc[split_index:]

split_index

709

In [148]:
x_train

,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_t
0,3,22.0,1,0.0,7.2500,0
1,1,38.0,1,0.0,71.2833,1
2,3,26.0,0,0.0,7.9250,1
3,1,35.0,1,0.0,53.1000,1
4,3,35.0,0,0.0,8.0500,0
...,...,...,...,...,...,...
704,1,42.0,0,0.0,26.2875,0
705,1,22.0,0,0.0,151.5500,1
706,3,4.0,1,1.0,15.2458,0
707,1,24.0,0,0.0,49.5042,1


6. Scale the columns using min-max scalers


In [149]:
for i in x_train.columns:
  min_value = x_train[i].min()
  max_value = x_train[i].max()
  x_train[i] = (x_train[i] - min_value) / (max_value - min_value)
  x_test[i] = (x_test[i] - min_value) / (max_value - min_value) #splitting test set with min and max value of train set to avoid data leakage


<ipython-input-149-79c8715ad0a5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[i] = (x_train[i] - min_value) / (max_value - min_value)
<ipython-input-149-79c8715ad0a5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[i] = (x_test[i] - min_value) / (max_value - min_value) #splitting test set with min and max value of train set to avoid data leakage


In [150]:
x_train

,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_t
0,1.0,0.268139,0.125,0.000000,0.014151,0.0
1,0.0,0.470032,0.125,0.000000,0.139136,1.0
2,1.0,0.318612,0.000,0.000000,0.015469,1.0
3,0.0,0.432177,0.125,0.000000,0.103644,1.0
4,1.0,0.432177,0.000,0.000000,0.015713,0.0
...,...,...,...,...,...,...
704,0.0,0.520505,0.000,0.000000,0.051310,0.0
705,0.0,0.268139,0.000,0.000000,0.295806,1.0
706,1.0,0.041009,0.125,0.166667,0.029758,0.0
707,0.0,0.293375,0.000,0.000000,0.096626,1.0


7. Print the shape of the train and test set  

In [151]:
# YOUR CODE HERE
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((709, 6), (178, 6), (709,), (178,))

# NN implementation (10 Marks)

1. Implement your NN method. To predict **each** point (query point) of the test set, you need to:


> a. Find the training point with the smallest **Euclidian** distance  with the query point


  
> b. Use as a prediction the target value of the point with the smallest distance
to the query point

2. Compute the accuracy


In [152]:
# YOUR CODE HERE
predtion_values=[]
for i in range(0,len(x_test)):
  cal_distance=np.sqrt(np.sum((x_test.iloc[i]-x_train)**2,axis=1))

  #for the given test point finding most similar point in the train set and finding the index of that point
  min_distance_index=y_train.iloc[cal_distance.sort_values().head(1).index.to_list()[0]]

  predtion_values.append(min_distance_index)


In [153]:
accuracy =np.mean(predtion_values == y_test)
print("Accuracy:", round((accuracy*100),1))

Accuracy: 75.8


# k-NN implementation (10 Marks)

1. Extend the previous implementation by using the average of the k closest neighbours as a prediction

2. Add the **Manhattan** and **Hamming** distance.


In [154]:
class knn:
  def __init__(self,neighbors,x_test,x_train):
    self.neighbors=neighbors
    self.x_test=x_test
    self.x_train=x_train

  def eucledian(self,index):
    self.euclidian_distance=np.sqrt(np.sum((self.x_test.iloc[index]-self.x_train)**2,axis=1))
    return self.euclidian_distance

  def manhattan(self,index):
    self.manhattan_distance=np.sum(abs(self.x_test.iloc[index]-self.x_train),axis=1)
    return self.manhattan_distance

  def hamming(self,index):
    self.hamming_distance=np.sum(self.x_test.iloc[index]!= self.x_train,axis=1)
    return self.hamming_distance

  #stores the target value of the nearest neighbors in a dataframe.
  def min_index_calculator(self,cal_distance):
    min_distance_index=cal_distance.sort_values().head(self.neighbors).index.to_list()
    pred=[]
    for i in min_distance_index:
      pred.append(y_train.iloc[i])
    df_pred = pd.DataFrame(pred, columns=['Values'])
    return df_pred

  def predict(self,metric):

    prediction_values=[]

    for i in range(0,len(x_test)):
      if metric=="euclidian":
        cal_distance=self.eucledian(i)
      elif metric=="manhattan":
        cal_distance=self.manhattan(i)
      else:
        cal_distance=self.hamming(i)

      df_pred=self.min_index_calculator(cal_distance)

      prediction_values.append(df_pred.value_counts().head(1).index.to_list()[0][0]) #appends the majority vote of neighbors.

    accuracy=np.mean(prediction_values == y_test) #calculates accuracy.

    return accuracy



2. Compute the accuracy

> a. Is it more accurate compared to the previous implementation?

In [155]:
#Calculating Accuracy when number of neighbors considered is 5
knn_object=knn(5,x_test,x_train)

print("Accuracy using following metrics:","\n")
print("Euclidian: ",round(knn_object.predict('euclidian')*100,1),"\n",
      "Manhattan",round(knn_object.predict('manhattan')*100,1),"\n",
      "Hamming",round(knn_object.predict('hamming')*100,1))


Accuracy using following metrics: 

Euclidian:  86.0 
 Manhattan 87.1 
 Hamming 82.6



**a. Is it more accurate compared to the previous implementation**

**-> Accuracy has improved from previous implementation as NN only consideres its nearest 1 neighbor.**

**->Using knn we can experiment on the number of neighbors to be considered and decide the best according to which k value gives us the best accuracy as while increasing the value of k in k-NN can have certain advantages, such as reducing noise sensitivity and simplifying the model, excessively high k values may lead to underfitting, decreased discriminative power, and increased computational complexity thus finding optimal k value is essential.**


3. Test your approach for k  values = [1, 3, 5, 7] and the three distance measures

> a. Which is the best combination?

> b. How can you improve the results?


In [156]:
# YOUR CODE HERE
neighbors=[1,3,5,7]

accuracy_euclidian={}
accuracy_manhattan={}
accuracy_hamming={}

for n in neighbors:

  knn_object=knn(n,x_test,x_train)


  accuracy_eu=knn_object.predict("euclidian")
  accuracy_euclidian[n]=round(accuracy_eu*100,1)

  accuracy_mh=knn_object.predict("manhattan")
  accuracy_manhattan[n]=round(accuracy_mh*100,1)

  accuracy_hm=knn_object.predict("hamming")
  accuracy_hamming[n]=round(accuracy_hm*100,1)

df_metrics = pd.DataFrame([accuracy_euclidian, accuracy_manhattan, accuracy_hamming], index=['Euclidian', 'Manhattan', 'Hamming'])

# Rename columns
df_metrics.columns = ['k=1', 'k=3', 'k=5', 'k=7']

df_metrics

,k=1,k=3,k=5,k=7
Euclidian,75.8,83.7,86.0,84.8
Manhattan,77.5,84.8,87.1,84.8
Hamming,74.7,82.0,82.6,83.1


**A) Which is the best combination?**

-> A best combination cannot be determined as with different samples of the same dataset we get different combination of metric to perform best for example if we shuffle our dataset with random_seed=0 we get k=5 and euclidian metric to perform the best whereas without sampling k=5 and manhattan performs best.

Thus cross validation and averaging out the result is necessary.

**B) How can you improve the results?**

-> Results can be improved calculating the correlation between the survived and other columns and using only those columns with highest correlation.

->Experimenting with other powerful ensemble algorithms like random forest which could better define the relation between survived and other columns hence increasing the accuracy.

**Reminder: You can not use any library, with the exception of pandas and NumPy. So scikit-learn is forbidden!**

## Marking Scheme
**Program Correctness:** Your program should work correctly on all inputs (including new datasets). Also, if there are any specifications about how the program should be written, or how the output should appear, those specifications should be followed.

**Readability:** Variables functions should have meaningful names. The code should be organised into functions/methods where appropriate. There should be an appropriate amount of white space so that the code is readable, and the indentation should be consistent.

**Documentation:** your code and functions/methods should be appropriately commented. However, not every line should be commented because that makes your code overly busy. Think carefully about where comments are added.


**Code Elegance:** There are many ways to write the same functionality into your code, and some of them are needlessly slow or complicated. For example, if you are repeating the same code, it should be inside creating a new method/function or for loop

**Code efficiency:** The implementation is logically well-designed without inappropriate
design choices (e.g., unnecessary loops).